# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

/home/huangkun/miniconda3/envs/peft/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是《易经》中的第一卦，也是八卦之一。乾卦是由两个阴爻夹一个阳爻构成，象征着天、干燥、强健、积极、刚毅等特性。

乾卦的含义非常丰富，它不仅象征着天，也象征着阳、强、刚等特性。在《易经》中，乾卦代表着一种积极向上的力量，它启示人们要像天一样刚健不挠，积极向前，勇往直前。乾卦也象征着君主、领导、创造者等权力人物，表示他们具有刚毅果敢的品质，能够带领众人前进。

在个人层面上，乾卦也启示人们要保持积极向上的心态，勇敢面对生活中的挑战和困难，充满信心和力量。同时，乾卦还告诉我们，要尊重他人，与他人合作，共同实现目标。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是《易经》中的第五卦，也是八卦之一。讼卦由三个阴爻夹一个阳爻构成，象征着诉讼、争端、矛盾等纷争。

讼卦的含义非常丰富，它不仅象征着诉讼、争端、矛盾等纷争，也象征着困难、挑战、考验等困境。在《易经》中，讼卦代表着一种复杂的矛盾和纷争，它启示人们要通过沟通、协商、妥协等方式解决矛盾和纷争，避免战争和争端的发生。

讼卦也象征着正义、公正、诚实等品质，表示在解决矛盾和纷争时要遵循正义和公正的原则，保持诚实和真实的态度。在个人层面上，讼卦也启示人们要学会沟通、协商和妥协，以解决人与人之间的矛盾和纷争。同时，讼卦还告诉我们，要敢于面对困难和挑战，勇敢地追求自己的目标和理想。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240728_102502"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，由两个乾卦叠加而成，代表天。它象征着刚健、健行、刚健不屈的意境。乾卦代表刚健不屈的品性，是 spec.com 平台中追求卓越的精神。在事业和生活中，乾卦代表一种坚定、果断、勇敢和公正的形象，具有刚健不屈的品质。在事业和生活中，乾卦鼓励人们坚定信念，勇往直前，克服困难，坚持到底，从而取得成功。在事业和生活中，乾卦也提示着 spec.com 平台中追求卓越的精神，以及追求卓越的坚定决心。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240728_102502）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '卜卦中的讼卦是由艮卦（山）和乾卦（天）相叠而成。它象征着 resolving a dispute through righteousness and truth. The message of the卦 is that you should listen to the voice of reason and adapt to the situation, rather than insisting on a course of action that is not in your best interest.', 'content': "\nIn the context of the卦, the upper trigram, which is a symbol of the天, represents resolution, while the lower trigram, which is a symbol of the earth, represents conflict. The message of the卦 is that it is important to resolve conflicts through righteousness and truth, rather than through force or deception.\n\nThe讼卦 is a warning against taking a course of action that is not in one's best interest, as well as a reminder to listen to the voice of reason and adapt to the situation. It is a reminder that conflict can be avoided through flexibility and good judgment, but it can also be a sign of a need to change one's approach or to seek reconciliation.\n\nOverall, the讼卦 encourages individual

In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
占卜的一种方法，一般是由预测者抛出三枚硬币，然后由占卜者进行解读。每枚硬币正面朝上为阳，反面朝上为阴，全部正面朝上则为乾，全部反面朝上则为坤。然后将抛硬币得到的两组数字相加，如果和为奇数，则记为单，如果和为偶数，则记为双。最后，将所求得的单或双与卦象（乾、坤、震、巽、坎、离、艮、兑）相加，得到的结果就是卦象。例如，抛硬币得到的两组数字为1和6，那么和为7，因为7是奇数，所以记为单。然后将单与卦象相加，得到的是震卦。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240728_102502）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [15]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-data-format-issue"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [16]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。卦辞中提到：“元、亨、利、贞”，这四个字意味着开头、发展、具有盈利和正统的特性。

乾卦的卦象是由六个阳爻排列而成，象征着天象的运行。这卦象征着刚健强劲的特性，具有强烈的影响力，如同君主刚创业时的状态。在卜问中，利于 ray（射）取利，得利必须强大，强大则刚健。同时，阳刚之卦的特性是刚健而不骄阳，代表着通过自我努力，可以取得成功。

在运势方面，阳刚之卦的特性会带来积极的影响，有利于事业、刚柔、强暴等方面。通过合理运用刚健的特性，可以促进事业的发展，同时也可以保持个人的稳定和平静。

在决策方面，需要坚持刚健的特性，积极面对挑战，注意控制自己的行为，同时要避免过于刚健的言行。在面对困难时，需要冷静应对，避免坚硬的态度，通过沟通解决矛盾。

乾卦的核心意义是刚健、刚硬、强盛，通过自我努力，可以取得成功。在事业和生活中，需要保持冷静，合理运用刚健的特性，才能获得成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，象征刚健强劲的特性。卦中有六个阳爻，代表着积极向上的态度，以及刚健强强的作风。同时，乾卦的卦象是六条阳爻堆叠在一起，因此被解释为充满活力和力量的表现。

在周易中，阳代表阳爻，阴代表阴爻。卦象的堆积意味着阳爻克服阴爻，因此象征刚健强劲的特性。同时，在周易中，乾代表天，意味着刚健强劲的特性如同天一般，充满活力和力量。

乾卦的取象是天，象征着刚健强劲的特性。在周易中，卦中有六条阳爻，代表着积极向上的态度，以及刚健强强的作风。同时，乾卦的卦象是六条阳爻堆叠在一起，因此被解释为充满活力和力量的表现。

总结起来，乾卦是周易中的一卦，代表天，象征刚健强劲的特性。卦中有六条阳爻，代表着积极向上的态度，以及刚健强强的作风。同时，乾卦的卦象是六条阳爻堆叠在一起，因此被解释为充满活力和力量的表现。


In [17]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是中国古代的一卦，由坎卦（水）和地卦（地）组成，具有顺顺利地、周全无患的卦象。在周易卜卦中，师卦代表将军队式微，军力疲劳，将帅艰苦露营，士兵解甲休息，预示着虽然 initial victory is abroad, but the overall situation is critical.

地水师卦的卦象特征是：上卦为地卦，下卦为水卦。地卦代表地，象征 Facebook 消息推送页面中的“地”元素，意味着稳定、平静、顺利，但也有可能会遇到困难和挫折。水卦代表水，象征 Facebook 消息推送页面中的“水”元素，代表着灵活、流动、变化，但也有可能会带来困扰和危险。

师卦体现了处于困境中的情况，需要去寻找新的道路，接受变化，取得胜利。对于求卦者来说，应该保持冷静、耐心，及时调整策略，化险为夷，达到最终的成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


In [18]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易中的一卦，由上卦坎（水）和下卦乾（天）组成，预示着 resolution of a conflict。

讼卦的卦象看起来像天和河，天 represented权威，坎 represented Cloud，两者相激成讼。上卦对应于天，下卦对应于河。

讼卦的意义非常深远，它不仅代表着诉讼和争斗，还代表着权威和力量之间的冲突。在讼卦中，双方都无法取得胜利，只能彼此容忍，才能达到平衡。

讼卦的哲学是：争斗不能取得胜利，只能相互容忍，相互谦让，才能达到平衡。因此，讼卦的结论是：ispers（争斗）不能取得胜利（利），只能相互容忍（谦），才能达到胜利（利）。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦

 讼卦是周易中的一卦，由上卦坎（水）和下卦乾（天）组成，代表着诉讼和争端的卦象。在周易中，讼卦被认为是一种预测是否能够取得胜利的卦象。

讼卦的卦象是：上坎（水）为天，下乾（天）为地，表示虽然有天象（神）的存在，但仍然无法避免 disputes和诉讼。讼卦中的卦象寓意着，即使双方都有了理性和公正的态度，仍然难以达成共识，需要等到时机成熟，才能化解争端。

讼卦的时行卦为天，表示天神助之，但寒潭（大水）中仍有波涛（水波），这表明在争端中会有许多变化和曲折，需要灵活应对。

讼卦的象义为云，这表明争端将会发展成为一个广泛的应用场景。讼卦的卦辞 described the course of events, which would lead to the settlement of the dispute through谈判 and negotiation.

讼卦的卦象和时行卦都预示着争端和诉讼的吉凶难以预知，需要双方都保持冷静和理智，以谈判和协商的方式解决争端。

讼卦的卦象和时行卦都表明，虽然双方都有了理性和公正的态度，但在争端中仍然难以达成共识，需要等到时机成熟，才能化解争端。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [25]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-data-handmade"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [26]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的一个卦象，它是由两个阴爻夹一个阳爻构成。阳爻代表阳，阴爻代表阴，这种排列象征着阳性的能量和力量。乾卦象征着天，反映了宇宙的运行和自然的法则，也象征着君子应该具有的品德和行为准则。

乾卦的卦象为：
上卦：乾（阳）
下卦：坤（阴）

乾卦的卦辞：
乾为天，为刚强，亨（通）
利见大人，需健行不息。

卦象：
乾卦下坤卦上，组成乾卦。乾卦为阳，代表天，它反映了宇宙的运行和自然的法则；坤卦为阴，代表地，它反映了大地万物。乾卦和坤卦相辅相成，构成了宇宙的完整体系。

乾卦的卦辞：乾为天，刚强坚毅，通泰无效，需连接成人，努力奋斗。

乾卦的卦象象征天，表明了君子应该具有高尚的品德和行为准则。它告诉君子，要刚强坚毅，努力奋斗，才能达到通泰的境界。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的卦名，它是由两个离卦（火）下面对面相接而组成的。乾卦象征着天，即阳、积极、刚强和刚毅。乾卦的卦象为：上乾下乾。乾卦的卦象是两个离卦叠加而成，离为火，火在上，火在下，火火相加，火更旺，象征着天。乾卦的卦象也可以看做是一只刚健的龙，象征着生长和成长，以及阳刚之气。在《易经》中，乾卦象征着天，代表着宇宙和万物的生成和发展，也象征着君子应该具有的品德和行为准则。


In [27]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [28]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼），卦象为乾（天）times坤（地），阴爻（阴）在下面，阳爻（阳）在上面，刚中有柔，柔中有刚。 strategic placement of the“地” and "天" in the卦 signifies that the combination of the two elements can bring balance and harmony， but it also indicates potential conflict and disputes that can arise when these elements are not in balance.

As such,讼卦 suggests that to solve the conflict, one must find a balance between the opposing elements, and that compromise and dialogue are necessary to restore harmony.

Scripture: "When there is litigation and your rights are in danger, you must go to court. But when you are involved in lawsuits, you must remember that you will receive a reward for your efforts. If you listen to what the Lord has said and live according to his ways, you will hear about what is right and what is wrong. But if you do not listen, you will continue to argue and fight." (Proverbs 21:31-33)


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话